In [ ]:
# write this to command line if tensorboard problem del /q %TMP%\.tensorboard-info\*

In [15]:
%load_ext tensorboard

In [1]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime
import shutil
from tensorboard.plugins.hparams import api as hp


from functions import one_hot_encoding
from functions import df_to_dataset

np.set_printoptions(precision=4)

## Consuming CSV Data

In [2]:
data = pd.read_csv("data/training_dataset_full")

In [3]:
#put all coordinate values into [0,1] range
data["xphase3"] = data["xphase3"]/816000
data["yphase3"] = data["yphase3"]/816000

In [4]:
#convert x,y coordinates from phase4 to 200m x 200m buckets
custom_bucket_array = np.linspace(0, 816000, 41)
data["cutx"] = pd.cut(data["xphase4"],custom_bucket_array)
data["cuty"] = pd.cut(data["yphase4"],custom_bucket_array)
data["bucket"] = data["cutx"].astype(str) + data["cuty"].astype(str)
data.drop(data.columns[0], axis = 1, inplace = True)
data.drop(["yphase4", "xphase4", "cutx", "cuty"], axis = 1, inplace = True)

#do one hot encoding for buckets
data = one_hot_encoding(data)
data.drop("bucket", axis = 1, inplace = True)

In [6]:
#divide dataset into train,val and test 
train, val = train_test_split(data, test_size = 0.1)
val, test = train_test_split(val, test_size = 0.3)

In [7]:
#make feature columns which will be input to a model
feature_columns = []

#numeric cols
feature_columns.append(tf.feature_column.numeric_column("xphase3"))
feature_columns.append(tf.feature_column.numeric_column("yphase3"))   

#create feature layers
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [9]:
#make tensorflow compatible dataset
train_ds = df_to_dataset(train, batch_size=256)
val_ds = df_to_dataset(val, shuffle=False, batch_size=8)
test_ds = df_to_dataset(test, shuffle=False, batch_size=8)

## hyperparameters

In [10]:
#define hyperameter grid
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64,128,256]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(["adam"]))
HP_LEARNING_RATE = hp.HParam("learning_rate", hp.Discrete([0.1,0.5]))
METRIC_ACCURACY = 'accuracy'


#write hyperparameter to log file so those can be visualised in tensorboard
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_LEARNING_RATE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [11]:
#define model, compile it, and train it
def train_test_model(hparams):
    model = tf.keras.Sequential([
      feature_layer,
      tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='linear'),
      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),  
      tf.keras.layers.Dense(64, activation='linear'),
      tf.keras.layers.Dense(32, activation='linear'),
      tf.keras.layers.Dense(850, activation = "softmax")
    ])
    
    optimizer = hparams[HP_OPTIMIZER]
    learning_rate = hparams[HP_LEARNING_RATE]
    if optimizer == "adam":
        optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
    
    
    model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics = ["accuracy"])
    
    logdir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    
    
    #works with epoch 2
    model.fit(train_ds, validation_data = val_ds, epochs=50, 
              callbacks=[tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ]  ) 
    
    _, accuracy = model.evaluate(test_ds)
    
    return accuracy
    


In [12]:
#function for running the model
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
    tf.summary.scalar(METRIC_ACCURACY, accuracy*2, step=1)

In [13]:
#run all models
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            for learning_rate in HP_LEARNING_RATE.domain.values:

                  hparams = {
                      HP_NUM_UNITS: num_units,
                      HP_DROPOUT: dropout_rate,
                      HP_OPTIMIZER: optimizer,
                      HP_LEARNING_RATE: learning_rate
                  }
                  run_name = "run-%d" % session_num
                  print('--- Starting trial: %s' % run_name)
                  print({h.name: hparams[h] for h in hparams})
                  run('logs/hparam_tuning/' + run_name, hparams)
                  session_num += 1

--- Starting trial: run-0
{'num_units': 64, 'dropout': 0.1, 'optimizer': 'adam', 'learning_rate': 0.1}
Epoch 1/70
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.


C:\Users\Ivan\AppData\Roaming\Python\Python38\site-packages\keras\backend.py:4846: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


27/35 [======================>.......] - ETA: 0s - loss: 10.1546 - accuracy: 0.0052WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'xphase3': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>, 'yphase3': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>}
Consider rewriting this model with the Functional API.
35/35 [==============================] - 2s 14ms/step - loss: 9.3330 - accuracy: 0.0075 - val_loss: 6.1245 - val_accuracy: 0.0103
Epoch 2/70
35/35 [==============================] - 0s 9ms/step - loss: 5.5485 - accuracy: 0.0333 - val_loss: 5.3133 - val_accuracy: 0.0514
Epoch 3/70
35/35 [==============================] - 0s 9ms/step - loss: 5.0836 - accuracy: 0.0566 - val_loss: 4.8456 - val_accuracy: 0.0602
Epoch 4/70
35/35 [==============================] - 0s 8ms/step - loss: 4.8393 - accuracy: 0.0624 - val_loss: 4.8283 - val_accuracy: 0.0881
Epoch 5/70
35/35 [==========================

In [17]:
#visualise parameter tuning results in tensorboard
%tensorboard --logdir logs

In [18]:
#clean logs
shutil.rmtree("./logs")